In [1]:
import pandas as pd
from assign_unique_ids_functions import *
pd.options.display.max_columns = 999
import numpy as np
uhc = ['UID', 'Unit', 'Date', 'Event']

In [361]:
um1 = pd.read_csv('../input/ase-units.csv.gz')
um1.dropna(thresh=4, axis=0, inplace=True)
um1.rename(columns={'Effective.Date' : 'Start.Date'}, inplace=True)
um1.dropna(subset=['Start.Date'],inplace=True)
um1.drop('ase-units_ID', inplace=True, axis=1)
um1.loc[um1['End.Date'] < um1['Start.Date'], 'End.Date'] = np.nan
um1.drop_duplicates(inplace=True)
um1.head()

,Unit,Start.Date,End.Date,UID
0,14.0,2007-03-29,NaN,1
2,15.0,2007-03-29,NaN,2
4,1.0,1976-08-19,1976-08-20,3
5,4.0,1971-01-16,1976-08-18,3
6,22.0,1970-06-15,1971-01-15,3


In [364]:
um1[(um1['End.Date'].notnull()) & (um1['Start.Date']>um1['End.Date'])]

,Unit,Start.Date,End.Date,UID


In [365]:
um2 = pd.read_csv('../input/all-sworn-units.csv.gz')
um2.dropna(thresh=4, axis=0, inplace=True)
um2.dropna(subset=['Start.Date'],inplace=True)
um2.drop('all-sworn-units_ID', inplace=True, axis=1)
um2.loc[um2['End.Date'] < um2['Start.Date'], 'End.Date'] = np.nan
um2.drop_duplicates(inplace=True)
um2.head()

,Unit,Start.Date,End.Date,UID
0,3.0,2016-06-05,NaN,1
1,14.0,2007-03-29,NaN,1
2,14.0,2007-03-29,2016-06-04,1
3,15.0,2007-03-29,NaN,2
5,1.0,1976-08-19,1976-08-20,3


In [366]:
um2[(um2['End.Date'].notnull()) & (um2['Start.Date']>um2['End.Date'])]

,Unit,Start.Date,End.Date,UID


In [367]:
uh = pd.concat([um1, um2]).drop_duplicates()

In [368]:
uh.head()

,Unit,Start.Date,End.Date,UID
0,14.0,2007-03-29,NaN,1
2,15.0,2007-03-29,NaN,2
4,1.0,1976-08-19,1976-08-20,3
5,4.0,1971-01-16,1976-08-18,3
6,22.0,1970-06-15,1971-01-15,3


In [369]:
prof = pd.read_csv('officer-profiles.csv.gz')[['UID', 'Resignation.Date']]

In [279]:
prof = prof[(prof['UID'].isin(um1['UID'])) & (prof['Resignation.Date'].notnull())]

In [371]:
uh = uh.merge(prof, on='UID', how='left')

In [370]:
'''prof.columns = ['UID', 'End.Date']
uh_ed = uh[uh['End.Date'].notnull()]
uh_ned = uh[uh['End.Date'].isnull()].drop('End.Date', axis=1)
uh_ned = uh_ned.merge(prof, on='UID', how='left')
uh = uh_ed.append(uh_ned).reset_index(drop=True)
uh.sort_values(['UID', 'Start.Date']).head()'''

In [375]:
uh.loc[(uh['Resignation.Date'].notnull()) & (uh['Start.Date'] > uh['Resignation.Date']), 'End.Date'] = np.nan

In [376]:
uh.loc[(uh['Resignation.Date'].notnull()) &
       (uh['End.Date'].isnull()) &
       (uh['Start.Date'] < uh['Resignation.Date']), 'End.Date'] =  uh.loc[(uh['Resignation.Date'].notnull()) &
       (uh['End.Date'].isnull()) &
       (uh['Start.Date'] < uh['Resignation.Date']), 'Resignation.Date']

In [379]:
uh.head()

,Unit,Start.Date,End.Date,UID,Resignation.Date
0,14.0,2007-03-29,NaN,1,NaN
1,15.0,2007-03-29,NaN,2,NaN
2,1.0,1976-08-19,1976-08-20,3,NaN
3,4.0,1971-01-16,1976-08-18,3,NaN
4,22.0,1970-06-15,1971-01-15,3,NaN


In [380]:
uh_kd = aggregate_data(keep_duplicates(uh, ['UID','Start.Date', 'Unit']) , uid='UID', id_cols=['Start.Date', 'Unit'], max_cols = ['End.Date'])

In [381]:
uh = remove_duplicates(uh,['UID','Start.Date', 'Unit']).append(uh_kd)

In [382]:
uh.sort_values(['UID', 'Start.Date']).head()

,End.Date,Resignation.Date,Start.Date,UID,Unit
0,2016-06-04,NaN,2007-03-29,1,14.0
108697,NaN,NaN,2016-06-05,1,3.0
1,NaN,NaN,2007-03-29,2,15.0
4,1971-01-15,NaN,1970-06-15,3,22.0
3,1976-08-18,NaN,1971-01-16,3,4.0


In [383]:
max(uh['Start.Date'].dropna())

'2016-10-09'

In [384]:
max(uh['End.Date'].dropna())

'2017-08-24'

In [386]:
uh.sort_values('End.Date', ascending=True, na_position='first').tail(5)

,End.Date,Resignation.Date,Start.Date,UID,Unit
856,2017-08-15,NaN,2006-06-13,8546,610.0
41863,2017-08-19,2017-08-19,1999-09-16,11359,2.0
48753,2017-08-19,2017-08-19,2015-01-15,13454,5.0
1335,2017-08-19,NaN,2000-06-22,13454,7.0
66639,2017-08-24,2017-08-24,2000-06-26,18928,2.0


In [391]:
pd.to_datetime(pd.date_range(start = max(uh['Start.Date'].dropna()), periods=2, freq='D'))[1]

Timestamp('2016-10-10 00:00:00', freq='D')

In [394]:
date_cap = pd.to_datetime(pd.date_range(start = max(uh['Start.Date'].dropna()), periods=2, freq='D'))[1].date()

In [395]:
date_cap

datetime.date(2016, 10, 10)

In [396]:
uh.loc[(uh['End.Date'].isnull()) | 
       (pd.to_datetime(uh['End.Date']) > date_cap),
  'End.Date'] = pd.to_datetime(date_cap)
uh['End.Date']= pd.to_datetime(uh['End.Date'])
uh['Start.Date']= pd.to_datetime(uh['Start.Date'])

In [397]:
uh.reset_index(drop=True, inplace=True)

In [398]:
uh.head()

,End.Date,Resignation.Date,Start.Date,UID,Unit
0,2016-10-10,NaN,2016-06-05,1,3.0
1,2016-10-10,NaN,2007-03-29,2,15.0
2,1971-01-15,NaN,1970-06-15,3,22.0
3,1976-08-18,NaN,1971-01-16,3,4.0
4,1976-08-20,NaN,1976-08-19,3,1.0


In [443]:
tuh = uh[['Start.Date', 'End.Date', 'UID', 'Unit']]

In [444]:
def expand_times(sd,ed,unit,uid):
    outdf = pd.DataFrame(pd.date_range(start=sd, end=ed, freq='D', name='Date'))
    outdf.insert(1, 'Unit', unit)
    outdf.insert(2, 'UID', uid)
    return outdf

In [446]:
import time
start = time.time()
full_df = pd.concat([expand_times(r['Start.Date'], r['End.Date'], r['Unit'], r['UID']) for i,r in tuh.iterrows()])
end = time.time()
print(end - start)

207.50347900390625


In [451]:
full_df.memory_usage() / 1e9

Index    1.924042
Date     1.924042
Unit     1.924042
UID      1.924042
dtype: float64

In [453]:
full_df = full_df[full_df['Date'] >= '2000-01-01']

In [454]:
full_df.memory_usage() / 1e9

Index    0.934583
Date     0.934583
Unit     0.934583
UID      0.934583
dtype: float64

In [455]:
full_df.head()

,Date,Unit,UID
0,2016-06-05,3.0,1
1,2016-06-06,3.0,1
2,2016-06-07,3.0,1
3,2016-06-08,3.0,1
4,2016-06-09,3.0,1


In [297]:
uhm = uh.copy()

In [298]:
uhm['End.Date'] = uhm['End.Date'].map(lambda x: x.replace(day=1))

In [299]:
uhm['Start.Date'] = pd.to_datetime(uhm['Start.Date']).map(lambda x: x.replace(day=1))

In [327]:
uhm.head()

,End.Date,Start.Date,UID,Unit,TID
0,2016-11-01,2016-06-01,1,3.0,1
1,2016-06-01,2007-03-01,1,14.0,1
2,2016-11-01,2007-03-01,2,15.0,2
3,1971-01-01,1970-06-01,3,22.0,3
4,1976-08-01,1971-01-01,3,4.0,3


In [341]:
pd.period_range('2016-06-01', '2016-11-01', periods=1).end_time

DatetimeIndex(['2016-06-01'], dtype='datetime64[ns]', freq=None)

In [359]:
uhm[uhm['Start.Date'] > uhm['End.Date']]

,End.Date,Start.Date,UID,Unit,TID
630,1981-09-01,1990-01-01,148,631.0,147
705,1986-03-01,1990-01-01,162,13.0,160
720,1981-06-01,1990-01-01,164,6.0,162
995,1986-06-01,1990-01-01,234,13.0,229
1127,1987-02-01,1990-01-01,271,11.0,266
1167,1981-03-01,1990-01-01,277,12.0,272
1173,1986-10-01,1990-01-01,279,1.0,274
1268,1983-09-01,1990-01-01,310,13.0,304
1432,1987-08-01,1990-01-01,341,12.0,334
1487,1986-08-01,1990-01-01,356,18.0,348


In [348]:
(pd.to_datetime('2016-11-01') - pd.to_datetime('2016-06-01')).days

153

In [351]:
pd.Period('2016-06-01', freq='153D').end_time

Timestamp('2016-10-31 23:59:59.999999999')

In [354]:
uhm['Start.Date'] = pd.to_datetime(uhm['Start.Date'])
uhm['End.Date'] = pd.to_datetime(uhm['End.Date'])

In [357]:
def make_period(start, end):
    freq = str((end-start).days + 1) + 'D'
    return pd.Period(start, freq=freq)

In [358]:
uhm.apply(lambda row: make_period(start = row['Start.Date'], end = row['End.Date']))

KeyError: ('Start.Date', 'occurred at index End.Date')